# ST7 Planification quotidienne d’une équipe mobile
# Phase III

## Importation de modules

In [122]:
# module importation
import numpy as np
from math import ceil
import matplotlib.pyplot as plt

# utilities
from utils import *
from copy import deepcopy

# model classes for employees and nodes
from models_v2 import Employee, Node, Task, Home, Unavail

Déclaration des constantes et des indices :

In [123]:
W = U = T = V = 0
employees = homes = tasks = unavails = nodes = []

## Fonction de lecture de données

In [124]:
def load_data_from_path(path_to_instance: str):
    # load employee data
    Employee.load_excel(path_to_instance)

    # load node data
    Node.clear_previous_data()
    for cls in [Home, Task, Unavail]:
        cls.load_excel(path_to_instance)
    Node.initialize_distance()

    # constants
    global W, U, T, V
    (W, U, T, V) = (Task.count, Unavail.count, Employee.count, Node.count)

    # indices of employees, homes, lunches, tasks, unavailabilities
    global employees, homes, tasks, unavails, nodes
    employees = list(range(T))
    homes = list(range(T))
    tasks = list(range(T, T + W))
    unavails = list((range(T + W, V)))
    nodes = list(range(V))

Lecture de données de test

In [125]:
path_to_test = path_bordeaux_v2 = "./data/InstancesV2/InstanceBordeauxV2.xlsx"
load_data_from_path(path_to_test)

## Implémentation de la classe Solution

In [252]:
class Chemins:
    print_warning = True  # whether to print warning or not when validating

    def __init__(self,chemins):
        self.list = chemins
    
    @classmethod
    def set_warning(cls, print_warning: bool):
        """set whether to print warning when validating data"""
        cls.print_warning = print_warning

    @classmethod
    def warn(cls, warning: str, print_color="yellow"):
        """
        Print a text in yellow if print_warning set to True
        :param warning: the warning message we want to print
        :param print_color: the color of the printed message, defaulted to yellow
        """
        if not cls.print_warning: return # does nothing if print_warning set to False
        correspondance = {
            "yellow": Colors.WARNING,
            "cyan": Colors.CYAN,
            "green": Colors.GREEN,
            "red": Colors.FAIL
        }
        color = correspondance[print_color] if print_color in correspondance.keys() else Colors.WARNING
        print(f"{color}Error: {warning}{Colors.NORMAL}")

    def copy(self):
        """Return a deep copy of the instance"""
        return deepcopy(self)

    def validate_format(self):
        """Verify that chemins and margin have the right format"""
        if type(self.list) == str :
            Chemins.warn(self.list)
            return False

        for k in self.list.keys():
            if k not in employees :
                Chemins.warn(f"{k} is not the index of an employee")
                return False
            for v in list(self.list[k].keys())[1:] : 
                if v not in tasks + unavails: 
                    Chemins.warn(f"{v} is not the index of a task or an unvavailability")
                    return False
        return True

    def validate(self):
        """
        validate the modifications
        :return: whether the constraints are verified
        """

        # verify that x, y and l contain only 0 and 1
        if not self.validate_format(): return False

        # C1, okay thank to the format 

        # C2
        visited = []
        for k in self.list.keys():
            for v in self.list[k].keys():
                if v in visited :
                    Chemins.warn("C2: Each task should be done by at most one employee.")
                    Chemins.warn(f"Condition violated by node {v} done by employees {k}.", "cyan")
                    return False
                visited.append(v)

        # C3.a and C3.b2
        for v in unavails :
            employee = Node.list[v].employee
            employee_idx = Employee.index_of(employee)
            if not v in self.list[employee_idx].keys():
                Chemins.warn("C3.a: Unavailabilities should be visited.")
                Chemins.warn(f"Condition violated by node {v}.", "cyan")
                return False
        for k in self.list.keys():
            if list(self.list[k].keys())[0] != k:
                Chemins.warn(f"C3.b: Each employee should visits their home.")
                Chemins.warn(f"Condition violated by employee {k}.", "cyan")
                return False

        # C4, okay thank to the format 

        # C5 + C6.b
        for k in self.list.keys():
            delta1 = 0
            delta2 = 0
            for v in self.list[k].keys(): 
                if v == k :
                    continue
                self.list[k][v][0] += delta1
                self.list[k][v][1] -= delta2

                opening, closing = Node.list[v].opening_time, Node.list[v].closing_time
                m = self.list[k][v][1] - opening
                if m<0 :
                    Chemins.warn(f"C5: A task should be worked on between its opening time and closing time")
                    Chemins.warn(f"Condition violated by task {v}", "cyan")
                    return False
                if  opening > self.list[k][v][0] :
                    delta1 += opening - self.list[k][v][0]
                    self.list[k][v][0] = opening
                
                m = closing - (self.list[k][v][0] + Node.list[v].duration)
                if m<0 :
                    Chemins.warn(f"C5: A task should be worked on between its opening time and closing time")
                    Chemins.warn(f"Condition violated by task {v}", "cyan")
                    return False
                if  closing < self.list[k][v][1] + Node.list[v].duration:
                    delta2 += (self.list[k][v][1] + Node.list[v].duration) - closing
                    self.list[k][v][1] = closing - Node.list[v].duration

                if v in unavails :
                    self.list[k][v][0] = opening
                    self.list[k][v][1] = opening

        # C6.a + 
        for k in self.list.keys():
            v = list(self.list[k].keys())[1]
            if v not in unavails :
                if self.list[k][v][0] < Employee.list[k].start_time :
                    Chemins.warn(f"C6.a: Employees' start times should be respected")
                    Chemins.warn(f"Condition violated by employee {k}", "cyan")
                    return False

        # C7
        # TODO: verify C7

        # C8
        # TODO: verify C8

        # C9
        for k in self.list.keys():
            v = list(self.list[k].keys())[-1]
            if self.list[k][v][1] + Node.list[v].duration + (Node.distance[v, k] / Employee.speed) > Employee.list[k].end_time :
                Chemins.warn("C9: An employee should have enough time to go back home.")
                Chemins.warn(f"Condition violated by employee {k} after task {v}", "cyan")
                return False

        # C10
        for k in self.list.keys() :
            Lunch_Break = False
            for i in range(len(self.list[k].keys())):
                v = list(self.list[k].keys())[i]
                t1,t2 = self.list[k][v]
                if t1<= 13*60 and t2>=13*60 and t2-t1>=60 :
                    if t1<=12*60:
                        t_end_lunch = 13*60
                    else :
                        t_end_lunch = t1+60
                    def rec(i,t):
                        if i==len(self.list[k].keys()):
                            return True
                        else :
                            v1 = list(self.list[k].keys())[i-1]
                            v2 = list(self.list[k].keys())[i]
                            t1,t2 = self.list[k][v2]
                            if v1 == k:
                                d = (Node.distance[v1,v2]/Employee.speed)
                            else :
                                d = Node.list[v1].duration + (Node.distance[v1,v2]/Employee.speed)
                            T = t + d
                            if T>t2:
                                return False
                            if T<=t1 :
                                return True
                            else :
                                return rec(i+1,T)
                    Lunch_Break = rec(i+1,t_end_lunch)
                    if Lunch_Break : break
                    
                    
            if not Lunch_Break :
                Chemins.warn("C10: Each employee must have the time to lunch")
                Chemins.warn(f"Condition violated by employee {k}", "cyan")
                return False

        # C12
        for k in self.list.keys():
            for i in range(len(self.list[k].keys())-1):
                v1 = list(self.list[k].keys())[i]
                v2 = list(self.list[k].keys())[i+1]
                if i == 0 and self.list[k][v1][0] + (Node.distance[v1, v2] / Employee.speed) > self.list[k][v2][1] :
                    Chemins.warn("C12: The traveling time between two nodes should be sufficient.")
                    Chemins.warn(f"Condition violated between node {v1} and {v2}", "cyan")
                    return False

        # C13
        for k in self.list.keys():
            for v in self.list[k].keys():
                if v in tasks and Employee.list[k].level < Node.list[v].level :
                    Chemins.warn("C13: An employee can only do tasks that they are able to do.")
                    Chemins.warn(f"Condition violated by employee {k} at node {v}", "cyan")
                    return False

        # Every condition is verified
        return True
    





# Demo
On peut instancier une nouvelle instance de Solution en appelant simplement le constructeur Solution().
L’instance s’adapte alors automatiquement aux données présentes dans les classes Employee et Node

La méthode validate permet de déterminer si notre solution vérifie les contraintes. Lorsque au moins une contrainte n’est pas vérifiée. Dans ce cas-là, un message d’erreur correspondant à la première violation de contrainte s’affiche.

In [127]:
solution_instance = Solution()
solution_instance.validate()

Error: C3.a: Unavailabilities should be visited.
Error: Condition violated by node [0]


False

Pour modifier une solution, on modifie directement les tableaux numpy stockés en attribut des instances.

In [128]:
solution_instance.x[0, 0] = 1
solution_instance.validate()

Error: A node can't be linked to itself.
Error: Condition violated by node 0.


False

Pour copier une instance, on utilise la méthode copy(), qui fait un deep-copy de l’instance.

In [129]:
solution_instance_copy = solution_instance.copy()

Lorsque l’on modifie une copie, la solution originale n’est pas modifiée.

In [130]:
solution_instance_copy.x[0, 0] = 2
solution_instance.x[0, 0] = 0
solution_instance.x[1, 0] = solution_instance.x[0, 2] = 1

In [131]:
solution_instance.validate()

Error: C1: The number of entrance of each node should be equal to the number of exit.
Error: Condition violated by node [1] with [0] entrance(s) and [1] exit(s).


False

In [132]:
solution_instance_copy.validate()

Error: x should contain binary variables, values other than 0 and 1 detected.


False

L'affichage des contraintes non vérifiées sont pour le debug, et on peut ne pas les afficher lorsque l’on exécute nos algorithmes méta-heuristiques.

In [133]:
Solution.set_warning(False) # this tells the solution class to not print warning messages
solution_instance.validate()

False

In [134]:
Solution.set_warning(True) # modify setting to print warning messages again
solution_instance.validate()

Error: C1: The number of entrance of each node should be equal to the number of exit.
Error: Condition violated by node [1] with [0] entrance(s) and [1] exit(s).


False

In [269]:
# Tabu Search

# Définition des objets

def add_time(routes):
    routes_timed = {}
    for k in range(len(routes)):
        routes_timed[k] = {}
        v0 = routes[k][0]
        routes_timed[k][v0] = Employee.list[k].start_time
        for i in range(1,len(routes[k])):
            v1 = routes[k][i-1]
            v2 = routes[k][i]
            if i == 1 :
                d = ceil(Node.distance[v1,v2]/Employee.speed)
            else :
                d = Node.list[v1].duration + ceil(Node.distance[v1,v2]/Employee.speed)
            routes_timed[k][v2] = routes_timed[k][v1] + d
        
        v = routes[k][-1]
        margin = Employee.list[k].end_time - (routes_timed[k][v] + Node.list[v].duration + ceil(Node.distance[v,v0]/Employee.speed))
        if margin <0 :
            return "Not Possible"
        else :
            for i in range(len(routes[k])):
                v = routes[k][i]
                routes_timed[k][v] = [routes_timed[k][v],routes_timed[k][v]+margin]
    return routes_timed


# Définition de la list Tabou

Tabu_list = {}

def update(Tabu_list, opp, iter, tabu_step):
    Tabu_list[iter] = opp
    if iter >= tabu_step :
        del Tabu_list[iter - tabu_step]

# Définition du voisinage

def Crossing(route1, route2, max_len_cross):
    Neighbors = []
    n1 = len(route1)
    n2 = len(route2)
    for l1 in range(max_len_cross+1):
        for l2 in range(max_len_cross+1): 
            if l1 == 0 and l2 ==0:
                continue
            for i in range(1,n1-l1):
                for j in range(1,n2-l2):
                    new_route1 = route1[:i]+route2[j:j+l2]+route1[i+l1:]
                    new_route2 = route2[:j]+route1[i:i+l1]+route2[j+l2:]
                    Neighbors.append((new_route1,new_route2))
    return Neighbors

def Crossing_alone(route, max_len_cross):
    Neighbors = []
    n = len(route)
    for l1 in range(max_len_cross+1):
        for l2 in range(max_len_cross+1): 
            if l1 == 0 and l2 ==0:
                continue
            for i in range(1,n-l1):
                for j in range(i+l1+1,n-l2-l1-i):
                    new_route = route[:i]+route[j:j+l2]+route[i+l1:j]+route[i:i+l1]+route[j+l2:]
                    if new_route not in Neighbors :
                        Neighbors.append(new_route)
    return Neighbors

def create_neighborhood(routes, max_len_cross):
    Neighborhood = []
    T = len(routes)
    for i in range(T):
        Neighbors = Crossing_alone(routes[i], max_len_cross)
        for route in Neighbors:
            new_routes = routes[:i]+[route]+routes[i+1:]
            if Chemins(add_time(new_routes)).validate() :
                Neighborhood.append(new_routes)
        for j in range(i+1,T):
            Neighbors = Crossing(routes[i], routes[j], max_len_cross)
            for (route1,route2) in Neighbors:
                new_routes = routes[:i]+[route1]+routes[i+1:j]+[route2]+routes[j+1:]
                if Chemins(add_time(new_routes)).validate() :
                    Neighborhood.append(new_routes)
    return Neighborhood

# Initialisation / 1ère solution

# Itérations


In [231]:
C = Chemins(chemins = add_time([[0,10,7,6,4,12],[1,3,5,11,9]]))
C.validate()

True

In [270]:
Chemins.set_warning(False)
create_neighborhood([[0,10,7,6,4,12],[1,3,5,11,9]],10)

[[[0, 7, 10, 6, 4, 12], [1, 3, 5, 11, 9]],
 [[0, 6, 10, 7, 4, 12], [1, 3, 5, 11, 9]],
 [[0, 7, 6, 10, 4, 12], [1, 3, 5, 11, 9]],
 [[0, 10, 7, 6, 4, 12], [1, 5, 3, 11, 9]]]